In [1]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from datetime import datetime
import string

^ We are importing all the necessary packages in order to execute this code.

In [2]:
sc = SparkContext(appName="PythonStreamingQueueStream")    

^ We are creating a SparkContext here.

In [3]:
sqlContext = SQLContext(sc)

^ And here SqlContext.

In [4]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load('Employees.csv')
df.show(5)

+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE| JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567| 17-06-03|AD_PRES| 24000|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568| 21-09-05|  AD_VP| 17000|          null|       100|           90|
|        102|       Lex|  De Haan| LDEHAAN|515.123.4569| 13-01-01|  AD_VP| 17000|          null|       100|           90|
|        103| Alexander|   Hunold| AHUNOLD|590.423.4567| 03-01-06|IT_PROG|  9000|          null|       102|           60|
|        104|     Bruce|    Ernst|  BERNST|590.423.4568| 21-05-07|IT_PROG|  6000|          null|       103|           60|
+-----------+----------+

^ We are reading a csv file "Employees.csv", using sqlContext and saving it into "df"

In [5]:
df.printSchema()

root
 |-- EMPLOYEE_ID: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: string (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: string (nullable = true)



^ As you can see in the schema of df, HIRE_DATE is of type string.

In [6]:
df1=df.withColumn("HIRE_DATE",(to_date(df["HIRE_DATE"],'dd-MM-yy').alias('HIRE_DATE')).cast(DateType()))
df1.show(5)

+-----------+----------+---------+--------+------------+----------+-------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER| HIRE_DATE| JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+----------+-------+------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567|2003-06-17|AD_PRES| 24000|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|2005-09-21|  AD_VP| 17000|          null|       100|           90|
|        102|       Lex|  De Haan| LDEHAAN|515.123.4569|2001-01-13|  AD_VP| 17000|          null|       100|           90|
|        103| Alexander|   Hunold| AHUNOLD|590.423.4567|2006-01-03|IT_PROG|  9000|          null|       102|           60|
|        104|     Bruce|    Ernst|  BERNST|590.423.4568|2007-05-21|IT_PROG|  6000|          null|       103|           60|
+-----------+---

^ In the above cell, we are selecting "HIRE_DATE" column in which the date representaion is in "dd-MM-yy", and performing to_date operations on it and casting it int DateType

In [7]:
df1.printSchema()

root
 |-- EMPLOYEE_ID: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: date (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: string (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: string (nullable = true)



^ When you observe in the schema, HIRE_DATE is of *date* type

In [8]:
def dateCustomFormat(dataFrame, col, date_format):
    data = dataFrame.withColumn(col,(to_date(dataFrame[col], date_format).alias(col)).cast(DateType()))
    return data

^ we are able to define a function, which take dataFrame, col, and date_format as arguments, and perform the to_date function on it to convert a string type column into a date type column.

In [9]:
df2 = dateCustomFormat(df,"HIRE_DATE","dd-MM-yy")
df2.show(5)

+-----------+----------+---------+--------+------------+----------+-------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER| HIRE_DATE| JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+----------+-------+------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567|2003-06-17|AD_PRES| 24000|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|2005-09-21|  AD_VP| 17000|          null|       100|           90|
|        102|       Lex|  De Haan| LDEHAAN|515.123.4569|2001-01-13|  AD_VP| 17000|          null|       100|           90|
|        103| Alexander|   Hunold| AHUNOLD|590.423.4567|2006-01-03|IT_PROG|  9000|          null|       102|           60|
|        104|     Bruce|    Ernst|  BERNST|590.423.4568|2007-05-21|IT_PROG|  6000|          null|       103|           60|
+-----------+---

^ we are passing df, "HIRE_DATE", and "dd-MM-yy" as paramaters. The returned data is stored into df2

In [10]:
df2.printSchema()

root
 |-- EMPLOYEE_ID: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: date (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: string (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: string (nullable = true)



^ HIRE_DATE is changed to date type

In [11]:
hire_date =hire_date = df.select("HIRE_DATE").rdd.flatMap(lambda x: x).collect()

In [12]:
type(hire_date)

list

In [13]:
print(hire_date)

['17-06-03', '21-09-05', '13-01-01', '03-01-06', '21-05-07', '25-06-05', '05-02-06', '07-02-07', '17-08-02', '16-08-02', '28-09-05', '30-09-05', '07-03-06', '07-12-07', '07-12-02', '18-05-03', '24-12-05', '24-07-05', '15-11-06', '10-08-07', '18-07-04', '10-04-05', '01-05-03', '10-10-05', '16-11-07', '16-07-05', '28-09-06', '14-01-07', '08-03-08', '20-08-05', '30-10-05', '16-02-05', '10-04-07', '14-06-04', '26-08-06', '12-12-07', '06-02-08', '14-07-03', '26-10-05', '12-02-06', '06-04-06', '17-10-03', '29-01-05', '15-03-06', '09-07-06', '01-10-04', '05-01-05', '10-03-05', '15-10-07', '29-01-08', '30-01-05', '24-03-05', '20-08-05', '30-03-06', '09-12-06', '23-11-07', '30-01-04', '04-03-04', '01-08-04', '10-03-05', '15-12-05', '03-11-06', '11-11-05', '19-03-07', '24-01-08', '23-02-08', '24-03-08', '21-04-08', '11-03-05', '23-03-06', '24-01-06', '23-02-07', '24-03-07', '21-04-08', '11-05-04', '19-03-05', '24-03-06', '23-04-06', '24-05-07', '04-01-08', '24-01-06', '23-02-06', '21-06-07', '03

In [18]:
hire_dates = list(map(lambda x: datetime.strptime(x,  "%d-%m-%y").strftime("%Y-%m-%d"), hire_date))
print(hire_dates)

['2003-06-17', '2005-09-21', '2001-01-13', '2006-01-03', '2007-05-21', '2005-06-25', '2006-02-05', '2007-02-07', '2002-08-17', '2002-08-16', '2005-09-28', '2005-09-30', '2006-03-07', '2007-12-07', '2002-12-07', '2003-05-18', '2005-12-24', '2005-07-24', '2006-11-15', '2007-08-10', '2004-07-18', '2005-04-10', '2003-05-01', '2005-10-10', '2007-11-16', '2005-07-16', '2006-09-28', '2007-01-14', '2008-03-08', '2005-08-20', '2005-10-30', '2005-02-16', '2007-04-10', '2004-06-14', '2006-08-26', '2007-12-12', '2008-02-06', '2003-07-14', '2005-10-26', '2006-02-12', '2006-04-06', '2003-10-17', '2005-01-29', '2006-03-15', '2006-07-09', '2004-10-01', '2005-01-05', '2005-03-10', '2007-10-15', '2008-01-29', '2005-01-30', '2005-03-24', '2005-08-20', '2006-03-30', '2006-12-09', '2007-11-23', '2004-01-30', '2004-03-04', '2004-08-01', '2005-03-10', '2005-12-15', '2006-11-03', '2005-11-11', '2007-03-19', '2008-01-24', '2008-02-23', '2008-03-24', '2008-04-21', '2005-03-11', '2006-03-23', '2006-01-24', '2007

In [19]:
type(hire_dates)

list

In [20]:
data = ['01/01/2012 01:00', '01/01/2012 02:00', '01/01/2012 03:00', '01/01/2012 04:00', '01/01/2012 05:00',
        '01/01/2012 06:00', '01/01/2012 07:00', '01/01/2012 08:00', '01/01/2012 09:00', '01/01/2012 10:00',
        '01/01/2012 11:00', '01/01/2012 12:00', '01/01/2012 13:00', '01/01/2012 14:00', '01/01/2012 15:00',
        '01/01/2012 16:00', '01/01/2012 17:00', '01/01/2012 18:00', '01/01/2012 19:00', '01/01/2012 20:00', 
        '01/01/2012 21:00', '01/01/2012 22:00', '01/01/2012 23:00', '02/01/2012 00:00', '04/01/2012 23:00']

^ we manually typed some dates in a list and assigned it to data variable

In [25]:
type(data)

list

In [21]:
dates = list(map(lambda x: datetime.strptime(x,  "%m/%d/%Y %H:%M").strftime("%Y-%m-%d %I:%M:%S"), data))
print(dates)

['2012-01-01 01:00:00', '2012-01-01 02:00:00', '2012-01-01 03:00:00', '2012-01-01 04:00:00', '2012-01-01 05:00:00', '2012-01-01 06:00:00', '2012-01-01 07:00:00', '2012-01-01 08:00:00', '2012-01-01 09:00:00', '2012-01-01 10:00:00', '2012-01-01 11:00:00', '2012-01-01 12:00:00', '2012-01-01 01:00:00', '2012-01-01 02:00:00', '2012-01-01 03:00:00', '2012-01-01 04:00:00', '2012-01-01 05:00:00', '2012-01-01 06:00:00', '2012-01-01 07:00:00', '2012-01-01 08:00:00', '2012-01-01 09:00:00', '2012-01-01 10:00:00', '2012-01-01 11:00:00', '2012-02-01 12:00:00', '2012-04-01 11:00:00']


In [22]:
type(dates)

list

In [23]:
data1 = list(map(lambda x: datetime.strptime(x,  "%m/%d/%Y %H:%M").strftime("%Y/%m/%d %I:%M:%S"), data))
print(data1)

['2012/01/01 01:00:00', '2012/01/01 02:00:00', '2012/01/01 03:00:00', '2012/01/01 04:00:00', '2012/01/01 05:00:00', '2012/01/01 06:00:00', '2012/01/01 07:00:00', '2012/01/01 08:00:00', '2012/01/01 09:00:00', '2012/01/01 10:00:00', '2012/01/01 11:00:00', '2012/01/01 12:00:00', '2012/01/01 01:00:00', '2012/01/01 02:00:00', '2012/01/01 03:00:00', '2012/01/01 04:00:00', '2012/01/01 05:00:00', '2012/01/01 06:00:00', '2012/01/01 07:00:00', '2012/01/01 08:00:00', '2012/01/01 09:00:00', '2012/01/01 10:00:00', '2012/01/01 11:00:00', '2012/02/01 12:00:00', '2012/04/01 11:00:00']


In [25]:
type(data1)

list

In [24]:
oldformat = df2["HIRE_DATE"]
datetimeobject = datetime.strptime(oldformat,'%d-%m-%y')

TypeError: strptime() argument 1 must be str, not Column